In [6]:
# ÉTAPE 1 : Le patch "monkey" est crucial !
# La bibliothèque steam utilise gevent pour l'asynchrone.
# Ce patch doit être fait AVANT presque tous les autres imports.
import steam.monkey
steam.monkey.patch_minimal()

In [11]:
from steam.client import SteamClient
from steam.client.cdn import CDNClient
from gevent import sleep # Pour les pauses asynchrones
import sys

# --- Configuration ---
APP_ID_TO_CHECK = 570  # CS2 - vous pouvez changer cet AppID pour tester d'autres jeux
# ---------------------

# Fonction pour formater les octets en GB
def format_gb(bytes_size):
    return f"{bytes_size / (1024**3):.2f} GB"

def main():
    client = SteamClient()
    
    print(f"Tentative de connexion anonyme à Steam...")
    try:
        # Se connecter en tant qu'utilisateur anonyme
        client.anonymous_login() 
    except Exception as e:
        print(f"Erreur lors de la connexion anonyme : {e}")
        return

    # Attendre que la connexion soit établie
    while not client.logged_on:
        print("En attente de connexion...")
        client.run_forever()
        if not client.logged_on:
             sleep(1)

    print(f"Connexion réussie !")
    if client.user and hasattr(client.user, 'steam_id'):
        print(f"SteamID : {client.user.steam_id}")
    else:
        print("Connexion anonyme établie")
    print(f"Initialisation du client CDN (Content Delivery Network)...")
    
    # Créer une instance du client CDN en utilisant notre client Steam
    try:
        cdn = CDNClient(client)
    except Exception as e:
        print(f"Erreur lors de l'initialisation du CDN : {e}")
        client.logout()
        return

    print(f"Récupération des informations de dépôts pour l'AppID : {APP_ID_TO_CHECK}...")
    
    try:
        # MÉTHODE AMÉLIORÉE : Utiliser get_app_depot_info() au lieu de get_manifests()
        # Cette méthode est plus efficace et fonctionne avec plus de jeux
        depot_info = cdn.get_app_depot_info(APP_ID_TO_CHECK)
        
    except Exception as e:
        print(f"\nERREUR: Impossible de récupérer les informations de dépôts pour {APP_ID_TO_CHECK}.")
        print(f"Détail : {e}")
        print("Cela peut arriver si l'AppID est incorrect ou si vous n'y avez pas accès.")
        client.logout()
        return

    if not depot_info:
        print(f"Aucune information de dépôt trouvée pour l'AppID {APP_ID_TO_CHECK}.")
        client.logout()
        return

    print(depot_info)
    print(f"Traitement des informations de dépôts...")
    print("Calcul de la taille de chaque dépôt...\n")

    grand_total_size = 0
    grand_total_download_size = 0
    depot_details = []

    # Boucler sur chaque dépôt
    for depot_id, depot_data in depot_info.items():
        if isinstance(depot_data, dict) and 'manifests' in depot_data:
            manifests = depot_data.get('manifests', {})
            if 'public' in manifests:
                public_manifest = manifests['public']
                size = int(public_manifest.get('size', 0))
                download_size = int(public_manifest.get('download', 0))
                
                if size > 0:
                    depot_details.append({
                        'depot_id': depot_id,
                        'size': size,
                        'download_size': download_size
                    })
                    grand_total_size += size
                    grand_total_download_size += download_size
                    
                    print(f"  > Dépôt {depot_id:<15} | Taille: {format_gb(size):<10} | Téléchargement: {format_gb(download_size)}")

    # Déconnexion propre
    client.logout()

    # --- Affichage final ---
    print("\n" + "="*80)
    print(f"Calcul terminé pour l'AppID : {APP_ID_TO_CHECK}")
    print(f"Nombre total de dépôts analysés : {len(depot_details)}")
    print(f"TAILLE TOTALE (tous dépôts confondus) : {format_gb(grand_total_size)}")
    print(f"TAILLE DE TÉLÉCHARGEMENT TOTAL : {format_gb(grand_total_download_size)}")
    print("="*80)
    
    # Retourner les données pour utilisation ultérieure
    return {
        'app_id': APP_ID_TO_CHECK,
        'total_size': grand_total_size,
        'total_download_size': grand_total_download_size,
        'depot_count': len(depot_details),
        'depot_details': depot_details
    }

if __name__ == "__main__":
    result = main()

Tentative de connexion anonyme à Steam...
Connexion réussie !
Connexion anonyme établie
Initialisation du client CDN (Content Delivery Network)...
Récupération des informations de dépôts pour l'AppID : 570...
{'228983': {'config': {'oslist': 'windows'}, 'depotfromapp': '228980', 'sharedinstall': '1'}, '228989': {'config': {'oslist': 'windows'}, 'depotfromapp': '228980', 'sharedinstall': '1'}, '228990': {'config': {'oslist': 'windows'}, 'depotfromapp': '228980', 'sharedinstall': '1'}, 'overridescddb': '1', 'markdlcdepots': '1', 'workshopdepot': '570', '373301': {'manifests': {'public': {'gid': '9017251831056252461', 'size': '4106788660', 'download': '2459672736'}}}, '381451': {'manifests': {'public': {'gid': '8940677437305767982', 'size': '2109416069', 'download': '940591936'}}}, '381452': {'manifests': {'public': {'gid': '3976523190930178667', 'size': '2125312046', 'download': '1259967328'}}}, '381453': {'manifests': {'public': {'gid': '5723651394550524659', 'size': '2099946682', 'down

In [12]:
# Test avec différents AppID
def test_multiple_app_ids():
    """Test avec plusieurs AppID pour voir les différences"""
    
    # Liste d'AppID à tester
    test_app_ids = [
        (730, "CS2"),
        (570, "Dota 2"), 
        (578080, "PUBG"),
        (1172470, "Apex Legends"),
        (271590, "GTA V")
    ]
    
    print("🧪 Test avec plusieurs AppID\n")
    
    for app_id, game_name in test_app_ids:
        print(f"🎮 Test de {game_name} (AppID: {app_id})")
        
        # Modifier l'AppID dans la configuration
        global APP_ID_TO_CHECK
        APP_ID_TO_CHECK = app_id
        
        try:
            result = main()
            if result:
                print(f"✅ {game_name}: {result['total_size'] / (1024**3):.2f} GB\n")
            else:
                print(f"❌ {game_name}: Aucune donnée récupérée\n")
        except Exception as e:
            print(f"❌ {game_name}: Erreur - {e}\n")
        
        print("-" * 50)

# Exécuter les tests
test_multiple_app_ids()


🧪 Test avec plusieurs AppID

🎮 Test de CS2 (AppID: 730)
Tentative de connexion anonyme à Steam...
Connexion réussie !
Connexion anonyme établie
Initialisation du client CDN (Content Delivery Network)...
Récupération des informations de dépôts pour l'AppID : 730...
{'depotdeltapatches': '0', 'overridescddb': '1', 'baselanguages': 'english,german,french,italian,koreana,spanish,schinese,tchinese,russian,thai,japanese,portuguese,polish,danish,dutch,finnish,norwegian,swedish,hungarian,czech,romanian,turkish,brazilian,bulgarian,greek,ukrainian,latam,vietnamese,indonesian', 'workshopdepot': '730', '228988': {'config': {'oslist': 'windows'}, 'depotfromapp': '228980', 'sharedinstall': '1'}, '228990': {'config': {'oslist': 'windows'}, 'depotfromapp': '228980', 'sharedinstall': '1'}, '732': {'config': {'oslist': 'windows'}, 'systemdefined': '1', 'manifests': {'public': {'gid': '6454031150813597364', 'size': '8', 'download': '64'}, 'csgo_legacy': {'gid': '6314304446937576250', 'size': '363675822',